In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd

In [25]:
# 处理方法 保存
# 1512 蛋白质 708 drug
df_train=pd.read_csv('./DTI.txt',header=None,sep=' ').T
interaction=np.array(df_train)
pos_row, pos_col = np.nonzero(interaction) 
print(len(pos_col))
print(df_train.shape)
1923/(708*1512)

1923
(708, 1512)


0.0017963635548381312

In [12]:
df_train.min()
df_train.max()

0    110451
1    110451
2         1
dtype: int64

In [4]:
# build rating matrix for datasets
\
rate_matrix=torch.zeros(user_size,item_size).cuda()
for row in df_train.itertuples():
	rate_matrix[row[1],row[2]]=1


# np.save(r'./'+dataset+r'/'+'rate_sparse.npy',rate_matrix.cpu().numpy())


Index(['10897', '1'], dtype='object')

In [19]:
df_train=pd.DataFrame(np.load('./DDI.npy'))
df_train

,0,1,2,3,4,5,6,7,8,9,...,2310,2311,2312,2313,2314,2315,2316,2317,2318,2319
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
interaction=np.array(df_train)
pos_row, pos_col = np.nonzero(interaction)
len(pos_row)
587200/(2320**2)

0.10909631391200951

In [11]:
np.save('./DDI.npy',rate_matrix.cpu().numpy())